In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [ ]:
# %%shell
# git clone https://github.com/allenai/scirepeval.git
# cd scirepeval
# conda create -n scirepeval python=3.8
# source activate scirepeval
# pip install --upgrade pip
# pip install -q -r requirements.txt

In [ ]:
!nvidia-smi

Sat Apr  8 08:38:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/allenai/scirepeval.git
%cd scirepeval
!pip install --upgrade pip
!pip install -q -r requirements.txt

Cloning into 'scirepeval'...
remote: Enumerating objects: 1500, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1500 (delta 239), reused 263 (delta 204), pack-reused 1066
Receiving objects: 100% (1500/1500), 14.17 MiB | 18.44 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
/content/scirepeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.7 MB/s

In [ ]:
import datasets

#training/validation/eval metadata
fos = datasets.load_dataset('allenai/scirepeval', 'fos')
# high_inf_cite = datasets.load_dataset('allenai/scirepeval', 'high_influence_cite')

{'train': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/fos/train.jsonl', 'val': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/fos/val.jsonl', 'test': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/test/fos/meta.jsonl'}


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating evaluation split:   0%|          | 0/68147 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/541218 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/67631 [00:00<?, ? examples/s]

Dataset scirepeval downloaded and prepared to /root/.cache/huggingface/datasets/allenai___scirepeval/fos/1.1.0/05b6b341c1750891a5539df5b3eb892babd18e8f299bfaba4735b87e20f6cf83. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
fos

DatasetDict({
    evaluation: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 68147
    })
    train: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 541218
    })
    validation: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 67631
    })
})

In [ ]:
# Finding no. of unique labels
import itertools

labels = list(itertools.chain.from_iterable(fos['train']['labels']))
label_texts = list(itertools.chain.from_iterable(fos['train']['labels_text']))

labels_dict = dict()
temp = set()

for label_text, label in zip(label_texts, labels):
    temp.add((label_text, label))

labels_dict = {label_text:label for label_text, label in list(temp)}
num_labels = len(labels_dict)

print(labels_dict)

{'Materials science': 15, 'Geography': 10, 'Economics': 6, 'Psychology': 21, 'History': 12, 'Physics': 19, 'Law': 13, 'Chemistry': 4, 'Business': 3, 'Philosophy': 18, 'Linguistics': 14, 'Medicine': 17, 'Environmental science': 9, 'Biology': 2, 'Computer science': 5, 'Sociology': 22, 'Engineering': 8, 'Mathematics': 16, 'Political science': 20, 'Art': 1, 'Agricultural and Food sciences': 0, 'Geology': 11, 'Education': 7}


In [ ]:
# *** NOT COMPLETE ***
from datasets import concatenate_datasets

def pick_balanced_dataset(dataset, num_papers_to_pick, num_labels):
  num_total_papers = len(dataset)
  bal_dict = dict()  

  for label in range(num_labels):
    fos_subj = dataset.filter(lambda example: label in example['labels'])
    subj_count = len(fos_subj)
    print("Subject count : {}".format(subj_count))
    small_fos_subj = fos_subj.shuffle(seed=42).select(range(round(subj_count/num_total_papers*num_papers_to_pick)))
    temp_dict = dict()

    temp_dict['data'] = small_fos_subj
    temp_dict['count'] = len(small_fos_subj)
    bal_dict[label] = temp_dict  
 
  print("Total papers picked : {}\n".format(num_papers_to_pick))

  bal_dataset = list()
  for label_text in labels_dict.keys():
    label = labels_dict[label_text]
    num_papers = bal_dict[label]['count']
    print("{} : {} ({:.2f}%)".format(label_text, num_papers, num_papers/num_papers_to_pick*100))
    bal_dataset.append(bal_dict[label]['data'])

  # Finally, concatenate datasets for each subject and then shuffle the contents
  bal_dataset = concatenate_datasets(bal_dataset).shuffle(seed=40)  
  return bal_dataset

In [ ]:
fos_train = fos['train']
fos_eval = fos['evaluation']
num_papers_to_pick = 20000

fos_small_train = pick_balanced_dataset(fos_train, num_papers_to_pick, num_labels)
fos_small_eval = pick_balanced_dataset(fos_eval, num_papers_to_pick, num_labels)

Subject count : 19822


  0%|          | 0/542 [00:00<?, ?ba/s]

Subject count : 3256


  0%|          | 0/542 [00:00<?, ?ba/s]

Subject count : 54948


...

Subject count : 3156


  0%|          | 0/69 [00:00<?, ?ba/s]

Subject count : 3722


  0%|          | 0/69 [00:00<?, ?ba/s]

Subject count : 839
Total papers picked : 20000

Materials science : 880 (4.40%)
Geography : 361 (1.80%)
Economics : 841 (4.21%)
Psychology : 1092 (5.46%)
History : 1587 (7.94%)
Physics : 2616 (13.08%)
Law : 213 (1.06%)
Chemistry : 1526 (7.63%)
Business : 312 (1.56%)
Philosophy : 270 (1.35%)
Linguistics : 72 (0.36%)
Medicine : 2562 (12.81%)
Environmental science : 1970 (9.85%)
Biology : 2030 (10.15%)
Computer science : 1328 (6.64%)
Sociology : 246 (1.23%)
Engineering : 1328 (6.64%)
Mathematics : 1298 (6.49%)
Political science : 926 (4.63%)
Art : 126 (0.63%)
Agricultural and Food sciences : 735 (3.67%)
Geology : 446 (2.23%)
Education : 1111 (5.56%)


In [ ]:
# fos_dict = dict()

# for label in range(num_labels):
#     fos_subj = fos_train.filter(lambda example: label in example['labels'])
#     fos_dict[label] = fos_subj

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

...

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

  0%|          | 0/542 [00:00<?, ?ba/s]

In [ ]:
# num_total_papers = len(fos_train)
# print("Total papers : {}\n".format(num_total_papers))

# for label_text in labels_dict.keys():
#     num_papers = len(fos_dict[labels_dict[label_text]])
#     print("{} : {} ({:.2f}%)".format(label_text, num_papers, num_papers/num_total_papers*100))

Total papers : 541218

Geography : 9933 (1.84%)
Psychology : 29582 (5.47%)
Political science : 25118 (4.64%)
Engineering : 36055 (6.66%)
Mathematics : 35177 (6.50%)
Chemistry : 41384 (7.65%)
Linguistics : 1839 (0.34%)
Law : 5656 (1.05%)
Physics : 71124 (13.14%)
Materials science : 23750 (4.39%)
Computer science : 35961 (6.64%)
Sociology : 6446 (1.19%)
Philosophy : 7198 (1.33%)
Biology : 54948 (10.15%)
Agricultural and Food sciences : 19822 (3.66%)
Education : 30086 (5.56%)
History : 43016 (7.95%)
Economics : 22788 (4.21%)
Business : 8354 (1.54%)
Medicine : 69507 (12.84%)
Geology : 11890 (2.20%)
Environmental science : 53551 (9.89%)
Art : 3256 (0.60%)


In [ ]:
fos_small_train[4]

{'doc_id': '32692408',
 'corpus_id': 32692408,
 'title': 'Transboundary smoke haze pollution in Malaysia: inpatient health impacts and economic valuation.',
 'abstract': 'This study assessed the economic value of health impacts of transboundary smoke haze pollution in Kuala Lumpur and adjacent areas in the state of Selangor, Malaysia. Daily inpatient data from 2005, 2006, 2008, and 2009 for 14 haze-related illnesses were collected from four hospitals. On average, there were 19 hazy days each year during which the air pollution levels were within the Lower Moderate to Hazardous categories. No seasonal variation in inpatient cases was observed. A smoke haze occurrence was associated with an increase in inpatient cases by 2.4 per 10,000 populations each year, representing an increase of 31 percent from normal days. The average annual economic loss due to the inpatient health impact of haze was valued at MYR273,000 ($91,000 USD).',
 'labels': [9],
 'labels_text': ['Environmental science']}

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

max_length = 400

def preprocess(dataset, tokenizer):
    
    dataset_ = dict()
    labels, texts = list(), list()

    for paper_dict in dataset:
        text = paper_dict['title'] + ' ' + paper_dict['abstract']
        for label in paper_dict['labels']:      # For each label of a paper, add a separate record to the dataset
            labels.append(label)
            texts.append(text)

    dataset_['label'] = labels
    dataset_['text'] = texts
    preprocessed_dataset = Dataset.from_dict(dataset_)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding=True, truncation=True, max_length=max_length)

    tokenized_dataset = preprocessed_dataset.map(tokenize_function, batched=True) 
    return tokenized_dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Whole dataset will take 52 hours on GPU, 10k random samples around 1 hr

# fos_small_train = fos['train'].shuffle(seed=42).select(range(10000))
# fos_small_eval = fos['evaluation'].shuffle(seed=42).select(range(1300))
train = preprocess(fos_small_train, tokenizer)
eval = preprocess(fos_small_eval, tokenizer)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file https://hug

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/33 [00:00<?, ?ba/s]

In [ ]:
eval[3]['label']

6

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=num_labels)

loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    

In [ ]:
!pip install --upgrade protobuf
!cp /usr/local/lib/python3.9/dist-packages/google/protobuf/internal/builder.py /content/scirepeval
!pip install protobuf==3.19.5
!mv /content/scirepeval/builder.py /usr/local/lib/python3.9/dist-packages/google/protobuf/internal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-4.22.1-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.5
    Uninstalling protobuf-3.19.5:
      Successfully uninstalled protobuf-3.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.10.0 which is incompatible.
tensorflow-metadata 1.13.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.
tensorboardx 2.5.1 requires protobuf<=3.20.1,>=3.8.0, but you have protobuf 4.22.1 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.
grpcio-status 1.48.2 requires grpcio>=1.48.2, but you have grpcio

In [ ]:
!pip install numpy==1.23

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.2
    Uninstalling numpy-1.23.2:
      Successfully uninstalled numpy-1.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.14 requires pytz>=2022.5, but you have pytz 2022.4 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.5 which is incompatible.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.10.0 which is incompatible.
tensorflow-hub 0.13.0 requires protobuf>=3.19.6, b

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# !pip install numpy --upgrade

In [ ]:
# %%shell
# source activate env
# python -m pip install huggingface_hub

# huggingface-cli login

In [ ]:
fos_tiny_train = pick_balanced_dataset(fos_train, 100, num_labels)
fos_tiny_eval = pick_balanced_dataset(fos_eval, 100, num_labels)

tiny_train = preprocess(fos_tiny_train, tokenizer)
tiny_eval = preprocess(fos_tiny_eval, tokenizer)

Subject count : 19822
Subject count : 3256
Subject count : 54948


Subject count : 8354
Subject count : 41384
Subject count : 35961
Subject count : 22788


Subject count : 30086
Subject count : 36055
Subject count : 53551
Subject count : 9933


Subject count : 11890
Subject count : 43016
Subject count : 5656
Subject count : 1839
Subject count : 23750
Subject count : 35177


Subject count : 69507
Subject count : 7198
Subject count : 71124
Subject count : 25118
Subject count : 29582


...

Subject count : 4525
Subject count : 2865
Subject count : 3784
Subject count : 4524
Subject count : 6714
Subject count : 1231
Subject count : 1519
Subject count : 5406
Subject count : 726


Subject count : 245
Subject count : 2998
Subject count : 4422
Subject count : 8730
Subject count : 919
Subject count : 8912
Subject count : 3156


Subject count : 3722
Subject count : 839
Total papers picked : 100

Materials science : 4 (4.00%)
Geography : 2 (2.00%)
Economics : 4 (4.00%)
Psychology : 5 (5.00%)
History : 8 (8.00%)
Physics : 13 (13.00%)
Law : 1 (1.00%)
Chemistry : 8 (8.00%)
Business : 2 (2.00%)
Philosophy : 1 (1.00%)
Linguistics : 0 (0.00%)
Medicine : 13 (13.00%)
Environmental science : 10 (10.00%)
Biology : 10 (10.00%)
Computer science : 7 (7.00%)
Sociology : 1 (1.00%)
Engineering : 7 (7.00%)
Mathematics : 6 (6.00%)
Political science : 5 (5.00%)
Art : 1 (1.00%)
Agricultural and Food sciences : 4 (4.00%)
Geology : 2 (2.00%)
Education : 6 (6.00%)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# !pip install numpy --upgrade
# !pip install Pillow==9.4
# !pip install evaluate

In [ ]:
!pip show numpy

Name: numpy
Version: 1.23.0
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/local/lib/python3.9/dist-packages
Requires: 
Required-by: adapter-transformers, albumentations, altair, arviz, astropy, autograd, blis, bokeh, chex, cmdstanpy, contourpy, cufflinks, cupy-cuda11x, cvxpy, datascience, datasets, db-dtypes, dopamine-rl, ecos, evaluate, fairseq, flax, folium, gensim, gym, h5py, holoviews, hyperopt, ImageHash, imageio, imbalanced-learn, imgaug, jax, jaxlib, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, opencv-contrib-python, opencv-python, opencv-python-headless, opt-einsum, optax, orbax, osqp, pandas, pandas-gbq, pandas-profiling, patsy, phik, plotnine, prophet, pyarrow, pycocotools, pyerfa, pymc, pytensor, python-louvain, pytorch-lightning, PyWavelets, qdldl, qudid

In [ ]:
# import numpy as np
# import evaluate

# metric = evaluate.load("accuracy")

In [92]:
a = np.array([1, 2, 3, 9, 6, 9, 3, 9])
b = np.array([3, 5, 6, 9, 3, 8, 7, 9])

np.array(a == b if )

array([False, False, False,  True, False, False, False,  True])

In [84]:
import numpy as np

def compute_accuracy(pred_eval):
    logits, labels = pred_eval
    
    print("Labels : \n\n")
    print(labels)
    # Convert the predictions and labels to NumPy arrays
    predictions = np.argmax(logits, axis=-1)

    print("Predictions : \n\n")
    print(predictions)
    labels = labels.flatten()

    # Compute the accuracy
    accuracy = np.mean(predictions == labels)
    cs_accuracy = np.mean()
    print(accuracy)

    return {"accuracy": accuracy}

In [85]:
tiny_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tiny_train,
    eval_dataset=tiny_eval,
    compute_metrics=compute_accuracy,
)

In [86]:
tiny_trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 154
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 60


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.397955,0.534884
2,No log,1.309415,0.558140
3,No log,1.286905,0.552326


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 172
  Batch size = 8


Labels : 


[12  9 15  4  9 17  3 20  2  4 17  5 16  7 22  5  8  2  4 17 19  3  9 10
  2 17  9 10  8 12  5  1 19  2 21  6  5  7  2  4 19 11  2 16 19 12  8  2
  4  7 17  8 17 16 17 19 19  0  9 10  6 17 18  2  5  4  9 13 17 19  2  4
 12  2  4 17 12  7  5  8 17 21  4  9  7  9  2 17  5  9 17 20  5  8 19 17
 19  9  2 21 19  2 17 17 20  6 12 17 21 19  2 21  2 17  6 12 21  6 16  0
  8  9  2  5  7 17  4  9 10 20  8 11  5 16 10 11  7 15 12 16 19 16  9 19
  2  4 17 15  2  4 17 19  5  2  9 17  2 17  0  6 12 20 17  2 21  8  0 15
 17 17 17 20]
Predictions : 


[12  9 15  9  9 17  3 20  4  4  4 16 16  7  7  7  7  2  2  2 15  3  9  9
 17 17 10 10  8 12  8  1 19  2  2  6 19 21  2  2  2  4 16 16 19  4 15  4
  4  7  7  8 21 19 17 19 19  0 11 11  6  4  4  2  2  4  4 13 17 19  4  4
 12 17 17 17 12  7  5  5 21 21  9  9  7  9 17 17  5 21 17 17  8  8 15  2
 19  9  2  2 19 17 17 17 17  6  6 21 21 19  2  2  2  2  6  6 21  6 16  0
 15  9  2  2  7  7  4 10 10 20  8 11 16 16 11 11 21  4 12  3 19  4  9  9
 17 17 1

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 172
  Batch size = 8


Labels : 


[12  9 15  4  9 17  3 20  2  4 17  5 16  7 22  5  8  2  4 17 19  3  9 10
  2 17  9 10  8 12  5  1 19  2 21  6  5  7  2  4 19 11  2 16 19 12  8  2
  4  7 17  8 17 16 17 19 19  0  9 10  6 17 18  2  5  4  9 13 17 19  2  4
 12  2  4 17 12  7  5  8 17 21  4  9  7  9  2 17  5  9 17 20  5  8 19 17
 19  9  2 21 19  2 17 17 20  6 12 17 21 19  2 21  2 17  6 12 21  6 16  0
  8  9  2  5  7 17  4  9 10 20  8 11  5 16 10 11  7 15 12 16 19 16  9 19
  2  4 17 15  2  4 17 19  5  2  9 17  2 17  0  6 12 20 17  2 21  8  0 15
 17 17 17 20]
Predictions : 


[12  9 15  9  9 17  3 20  4  4  4 16 16  7  7  7  7  4  4  4 15  3  9  9
 17 17 10 10  8 12  8  1 19  2  2  6  9 21  2  2  2  4 16 16 19  4 15  4
  4  7  7  8 17 19 17 19 19  0  9  9  6 17 17  2  2  4  4 13 17 19  4  4
 12  4  4  4 12  7  5  5 21 21  9  9  7  9 17 17  5 21 17 17  8  8 15  2
 19  9  2  2 19 17 17 17 17  6  6 21 21 19  2  2  2  2  6  6 21  6 16  0
 15  9  5  5  7  7  4 10 10 20  8 11 16 16 11 11 21 15 12  3 19  4  9  9
  4  4  

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 172
  Batch size = 8


Labels : 


[12  9 15  4  9 17  3 20  2  4 17  5 16  7 22  5  8  2  4 17 19  3  9 10
  2 17  9 10  8 12  5  1 19  2 21  6  5  7  2  4 19 11  2 16 19 12  8  2
  4  7 17  8 17 16 17 19 19  0  9 10  6 17 18  2  5  4  9 13 17 19  2  4
 12  2  4 17 12  7  5  8 17 21  4  9  7  9  2 17  5  9 17 20  5  8 19 17
 19  9  2 21 19  2 17 17 20  6 12 17 21 19  2 21  2 17  6 12 21  6 16  0
  8  9  2  5  7 17  4  9 10 20  8 11  5 16 10 11  7 15 12 16 19 16  9 19
  2  4 17 15  2  4 17 19  5  2  9 17  2 17  0  6 12 20 17  2 21  8  0 15
 17 17 17 20]
Predictions : 


[12  9 15  9  9 17  3 20  4  4  4 16 16  7  7  7  7  2  2  2 15  3  9  9
 17 17 10 10  8 12  8  1 19  2  2  6  9 21  2  2  2  4 16 16 19  4 15  4
  4  7  7  8 17 19 17 19 19  0 11 11  6 17 17  2  2  4  4 13 17 19  2  2
 12  2  2  2 12  7  5  5 21 21  8  8  7  9 17 17  5 21 17 17  5  5 15 17
 19  9  2  2 19 17 17 17 17  6  6 21 21 19  2  2  2  2  6  6 21  6 16  0
 15  9  5  5  7  7  4 10 10 20  8 11 16 16 11 11 21 15 12  3 19  4  9  9
 17 17 1



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=60, training_loss=0.399067751566569, metrics={'train_runtime': 48.7519, 'train_samples_per_second': 9.477, 'train_steps_per_second': 1.231, 'total_flos': 94984552555200.0, 'train_loss': 0.399067751566569, 'epoch': 3.0})

In [87]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=compute_accuracy,
)

In [88]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 32152
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12057


Epoch,Training Loss,Validation Loss,Accuracy
1,0.697500,0.988886,0.620837


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

Labels : 


[ 2 16  5 ... 20  4  5]
Predictions : 


[16 16  5 ... 20  4  5]
0.6208369714356992


Saving model checkpoint to test_trainer/checkpoint-4500
Configuration saved in test_trainer/checkpoint-4500/config.json
Model weights saved in test_trainer/checkpoint-4500/pytorch_model.bin


KeyboardInterrupt: ignored